##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Keras

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Keras 是一款意于快速搭建训练 DeepLearning 模型的 High-level API. 
Keras 有三个关键优势: 

- *User friendly*<br>
  Keras 针对通用案例, 有简单,保持一致性的接口. 提供了清晰的 error 反馈.
- *模块化 & 组件化*<br>
  Keras 模型由一个个可配置的blocks 组成, 只有极少的限制, 如 LEGO 一般简单. 
- *便于扩展*<br> 
  通过编写自定义 blocks 来实现新的研究思路. 如创建新的 layers, loss functions, 和开发最新的 models. 

## Import tf.keras

`tf.keras` 是 TensorFlow 版的
[Keras API specification](https://keras.io){:.external}. 这个 high-level
API 除了原始 Keras 内容外, 还包括了一些
易用的TensorFlow-specific 功能, 比如 [eager execution](#eager_execution),
`tf.data` pipelines, 还有 [Estimators](./estimators.md).
`tf.keras` 让 TensorFlow 更加易用, 并且无需再灵活性和性能上有所牺牲. 

那么我们从 import `tf.keras` 来开启你的 TensorFlow program:

In [0]:
!pip install -q pyyaml  # pyyaml is an optional

In [0]:
import tensorflow as tf
from tensorflow.keras import layers
import keras 


print(tf.VERSION)
print(tf.keras.__version__)
print(keras.__version__)

`tf.keras` 可以运行任何 Keras-compatible 代码, 但是要记住:

* TensorFlow 内 `tf.keras`的最新 version 并不一定和 `keras` 的最新version 相同 . 通过 `tf.keras.__version__` 查看 TensorFlow 的 keras version
*  当 [保存一个 model 的 weights时](#weights_only), `tf.keras` 默认保存格式是
  [checkpoint format](./checkpoints.md). 通过 `save_format='h5'` 参数来使用 HDF5 格式. 

## 创建一个简单的 model

### Sequential model

使用Keras, 通过组装 *layers* 来创建一个 *models*. 一个 model 通常就是一系列 layers. 最常用的 model 类型就是一堆叠起来的 layers: 
`tf.keras.Sequential` 模型.

要创建一个简单的, fully-connected 网络 (例如, multi-layer perceptron):

In [0]:
model = tf.keras.Sequential()
# 添加一个包含64个单元的 densely-connected layer:
model.add(layers.Dense(64, activation='relu'))
# 再添加一层:
model.add(layers.Dense(64, activation='relu'))
# 添加一个包含10个输出单元的 softmax 层, 用于将数据分成10类:
model.add(layers.Dense(10, activation='softmax'))

### 配置 layers

在 `tf.keras.layers` 有很多可以使用的layer 类型, 它们有一些通用的构造参数 :

* `activation`: 为当前层设置激活函数. 此参数可以是内置的函数名称或可被调用的对象. 默认不指定任何激活函数.
* `kernel_initializer` 和 `bias_initializer`: layer 的 weights(kernel 和 bias) 的初始化方式. 参数可以是一个名称或可调用对象. 默认使用 `"Glorot uniform"` 初始化.
* `kernel_regularizer` and `bias_regularizer`: layer 的 weights (kernel and bias)的正则化方式 , 例如 L1 or L2
  正则化. 默认不使用正则化.

下面是使用构造函数初始化的 `tf.keras.layers.Dense` layers

In [0]:
# 创建一个 sigmoid layer:
layers.Dense(64, activation='sigmoid')
# 也可以这样:
layers.Dense(64, activation=tf.sigmoid)

# 一个对 kernal 矩阵使用0.01因子的L1 正则的 linear layer:
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# 一个对 bias 向量使用0.01因子的L2 正则的 linear layer:
layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# 一个对 kernal 矩阵使用随机正交矩阵初始化的 linear layer:
layers.Dense(64, kernel_initializer='orthogonal')

# 一个对 bias 向量使用常量2.0初始化的 linear layer:
layers.Dense(64, bias_initializer=tf.keras.initializers.constant(2.0))

## Train 和 evaluate

### 创建 training

在 model 构造好以后, 使用 `compile` 来配置学习过程: 

In [0]:
model = tf.keras.Sequential([
# 添加一个包含64个单元的 densely-connected layer:
layers.Dense(64, activation='relu'),
# 再添加一层:
layers.Dense(64, activation='relu'),
# 添加一个包含10个输出单元的 softmax 层, 用于将数据分成10类:
layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

`tf.keras.Model.compile` 包含三个主要参数:

* `optimizer`: 用于指定训练过程. optimizer在`tf.train` 模块下 ,  例如
  `tf.train.AdamOptimizer`, `tf.train.RMSPropOptimizer`, 或
  `tf.train.GradientDescentOptimizer`.
* `loss`: 在优化期间的最小化损失的检验计算方法. 通常使用
  mean square error (`mse`), `categorical_crossentropy`, 和
  `binary_crossentropy`. 损失函数通过`tf.keras.losses` 模块内的名称指定或传递一个可被调用对象.
* `metrics`: 用于监控训练. 通常通过`tf.keras.metrics` 模块内的名称或函数. 

下面是 model 配置的例子:

In [0]:
# 配置一个使用mean-squared error 的回归模型. 
model.compile(optimizer=tf.train.AdamOptimizer(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

# 配置一个用于分类的模型.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy])

### 输入 NumPy 数据

对于小数据集, 使用 [NumPy](https://www.numpy.org/){:.external}
数组来 train 和 evaluate 一个模型. 使用`fit` 函数来训练模型:

In [0]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.fit(data, labels, epochs=10, batch_size=32)

`tf.keras.Model.fit` 包含三个主要参数:

* `epochs`: 训练结构为 *epochs*. 一个 epoch 代表整个输入数据的一次循环 (上面的 model 会用小一些的batches来完成).
* `batch_size`: 当传入 NumPy 数据时, 模型将数据分割成多个小的 batch, 并在训练时循环每个小的 batch 内的数据. 这个整数指定了每个 batch 的大小. 需要注意的是, 当 batch_size 不能被数据数量整除时, 最后一个 batch 会小于这个指定值.
* `validation_data`: 当构建一个模型的原型时, 你想要基于某些验证数据来检验模型的好坏. 传入一组 inputs 和 lables 的 tuple 数据, 就可以在每个 epoch 结束时输出模型的 loss 和 metrics 了. 

下面是一个使用`validation_data`的例子:

In [0]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

model.fit(data, labels, epochs=10, batch_size=32,
          validation_data=(val_data, val_labels))

### 输入 tf.data 数据集

使用 [Datasets API](./datasets.md) 来扩展至大数据集或多设备训练.  向 `fit`输入 `tf.data.Dataset`参数:

In [0]:
# 使用假数据来初始化数据集:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32) # 指定了 batch_size
dataset = dataset.repeat()

# 不要忘记指定 `steps_per_epoch`
model.fit(dataset, epochs=10, steps_per_epoch=30)

这里, `fit` 函数使用了 `steps_per_epoch` 参数—这个参数指定了每个 epoch 的训练步数. 因为
`Dataset` 会产生 batches , 这个代码片段就不需要 `batch_size`了.

Datasets 同样可以用于 validation:

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32).repeat()

val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset = val_dataset.batch(32).repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset,
          validation_steps=3)

### 评估 和 预测

`tf.keras.Model.evaluate` 和 `tf.keras.Model.predict` 函数可以同时使用 NumPy 数据和 `tf.data.Dataset`.

使用已有数据 *评估*  inference-mode loss 和 metrics :

In [0]:
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

print(model.evaluate(data, labels, batch_size=32))

print(model.evaluate(dataset, steps=30))

最后一层的*预测*结果会输出一个 Numpy 数组: 

In [0]:
result = model.predict(data, batch_size=32)
print(result.shape)

## 构建高级模型

### Functional API

`tf.keras.Sequential` 模型只是层的堆栈, 无法用来实现灵活的模型. 使用
[Keras functional API](https://keras.io/getting-started/functional-api-guide/){:.external}
来购将更负责的模型, 例如:

* 多输入模型,
* 多输出模型,
* 使用共享层的模型 (同一层被多次调用),
* 使用非时序数据流的模型 (例如. residual connections).

使用 functional API 构建模型的步骤:

1. 层实例可以被调用并返回一个 tensor. 
2. 输入的 tensors 和输出的 tensors 用来定义`tf.keras.Model`实例.
3. 模型的训练部分与 `Sequential`模型相同.

下面用 functional API 构建一个简单的 fully-connected 网络: 

In [0]:
inputs = tf.keras.Input(shape=(32,))  # 返回一个tensor占位符 

# 一个层实例可以通过 tensor 调用, 并返回一个 tensor.
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x)

初始模型的输入和输出.

In [0]:
model = tf.keras.Model(inputs=inputs, outputs=predictions)

# compile 步骤, 指定训练的配置. 
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 训练5个循环
model.fit(data, labels, batch_size=32, epochs=5)

### Model 子类化

通过创建`tf.keras.Model`的子类来构建完全自定义的模型, 包括自定义前传. 在 `__init__` 函数内创建 layer 将他们设置为类的属性. 在 `call` 函数内定义前传.

当启用[eager execution](./eager.md)时, Model 子类化会更有用, 因为前传可以重写.


关键点: 使用正确的 API.  虽然模块子类化提供了更多的灵活性, 同时它也会带来更高的复杂度和更高的用户错误率. 如果可能, 推荐使用functional API.

下面的例子是一个使用了自定义前传的子类化`tf.keras.Model`:

In [0]:
class MyModel(tf.keras.Model):

  def __init__(self, num_classes=10):
    super(MyModel, self).__init__(name='my_model')
    self.num_classes = num_classes
    # 在这里自定义 layer.
    self.dense_1 = layers.Dense(32, activation='relu')
    self.dense_2 = layers.Dense(num_classes, activation='sigmoid')

  def call(self, inputs):
    # 这里使用在 `__init__` 函数内定义的层来自定义前传
    x = self.dense_1(inputs)
    return self.dense_2(x)

  def compute_output_shape(self, input_shape):
    # 如果你想将自己定义的 model 子类作为 functional-style 模型的一部分, 你需要重写这个方法
    # 如果不想, 则不需要重写
    shape = tf.TensorShape(input_shape).as_list()
    shape[-1] = self.num_classes
    return tf.TensorShape(shape)

实例化新的模型类:

In [0]:
model = MyModel(num_classes=10)

# compile 步骤, 指定训练的配置. 
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 训练5个 epoch.
model.fit(data, labels, batch_size=32, epochs=5)

### 自定义 layers

通过子类化`tf.keras.layers.Layer`来创建自定义 layer, 需要实现以下方法:

* `build`: 创建 layer 的 weights. 通过`add_weight`方法添加 weights.
* `call`: 定义前传.
* `compute_output_shape`: 指定如何通过输入的 shape 来计算输出的 shape. 
* 如果有需要, layer 可以通过实现`get_config` 方法和 `from_config` 方法来序列化.

下面是一个对输入做了 `matmul`处理, 和添加了 kernel matrix 的自定义 layer 例子:  

In [0]:
class MyLayer(layers.Layer):

  def __init__(self, output_dim, **kwargs):
    self.output_dim = output_dim
    super(MyLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    shape = tf.TensorShape((input_shape[1], self.output_dim))
    # 为自定义 layer, 创建一个可以被训练的 weight 变量.
    self.kernel = self.add_weight(name='kernel',
                                  shape=shape,
                                  initializer='uniform',
                                  trainable=True)
    # Be sure to call this at the end
    super(MyLayer, self).build(input_shape)

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

  def compute_output_shape(self, input_shape):
    shape = tf.TensorShape(input_shape).as_list()
    shape[-1] = self.output_dim
    return tf.TensorShape(shape)

  def get_config(self):
    base_config = super(MyLayer, self).get_config()
    base_config['output_dim'] = self.output_dim
    return base_config

  @classmethod
  def from_config(cls, config):
    return cls(**config)

通过自定义层创建一个模型:

In [0]:
model = tf.keras.Sequential([
    MyLayer(10),
    layers.Activation('softmax')])

# compile 步骤, 指定训练的配置. 
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 训练 5 个循环.
model.fit(data, labels, batch_size=32, epochs=5)

## 回调

通过给 model 传一个回调, 可以在训练时自定义或扩展模型. 可以自定义回调, 或使用内置的 `tf.keras.callbacks`:

* `tf.keras.callbacks.ModelCheckpoint`: 定期保存模型 checkpoint.
* `tf.keras.callbacks.LearningRateScheduler`: 动态改变学习率.
* `tf.keras.callbacks.EarlyStopping`: 当验证结果不再提升时, 中断训练. 
* `tf.keras.callbacks.TensorBoard`: 使用 [TensorBoard](./summaries_and_tensorboard.md). 监控. 

想要使用回调 `tf.keras.callbacks.Callback`, 只需要将回调传给模型的`fit` 函数即可: 

In [0]:
callbacks = [
  # 如果 `val_loss` 在2个循环内不再提升, 则中断训练
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  # 将 TensorBoard logs 写入 `./logs` 目录
  tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(data, labels, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(val_data, val_labels))

## 保存save和恢复restore

### 只保存 Weights 

通过 `tf.keras.Model.save_weights` 保存和加载一个模型的 weights:

In [0]:
model = tf.keras.Sequential([
layers.Dense(64, activation='relu'),
layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
# 将 weights 存储为一个 TensorFlow Checkpoint 文件
model.save_weights('./weights/my_model')

# 恢复模型状态,
# 需要保存的模型与新创建的模型由同样的结构architecture.
model.load_weights('./weights/my_model')

默认情况下, 模型的 weights 保存为 [TensorFlow checkpoint](./checkpoints.md) 文件格式. Weights 同样可以保存为 Keras HDF5 格式 ( Keras的默认格式):

In [0]:
# 保存 weights 为 HDF5 文件格式
model.save_weights('my_model.h5', save_format='h5')

# 恢复模型状态
model.load_weights('my_model.h5')

### 只保存配置

一个模型的配置也可以被保存, 保存格式是一个序列化的模块结构, 不包含任何 weights. 即使没有原始模型的代码, 被保存的配置同样可以被重新构建成一个一样的模型. Keras 同时支持 JSON 和 YAML 两种序列化格式: 

In [0]:
# Serialize a model to JSON format
json_string = model.to_json()
with open('model.json', 'w') as f:
   f.write(json_string)

In [0]:
import json
import pprint
pprint.pprint(json.loads(json_string))

通过 json 文件, 重新构建模型

In [0]:
fresh_model = tf.keras.models.model_from_json(json_string)

将模型序列化为 YAML 格式需要再你 `import TensorFlow` 前安装 `pyyaml` :

In [0]:
yaml_string = model.to_yaml()
print(yaml_string)

通过 yaml 重新构建模型.

In [0]:

fresh_model = tf.keras.models.model_from_yaml(yaml_string)

注意: 子类化的模型是不能被序列化的. 因为他们的结构是通过python 内的 `call` 函数来定义的. 但是可以序列化自定义 layer(通过实现 get_config 和 from_config)


### 全部保存

模型可以全部保存到一个文件内, 包括 weight 数值, 模型配置, 甚至 optimizer 配置. 模型可以训练中保存一个 checkpoint, 然后在其它时间从这个点上恢复后继续训练. 恢复时不需要任何原始代码. 

In [0]:
# 创建一个模型
model = tf.keras.Sequential([
  layers.Dense(10, activation='softmax', input_shape=(32,)),
  layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels, batch_size=32, epochs=5)


# 将整个模型保存为 HDF5 格式文件
model.save('my_model.h5')

# 创建相同的模型, 包括 weights 和 optimizer.
model = tf.keras.models.load_model('my_model.h5')

## Eager execution

[Eager execution](./eager.md) 是一个命令式变成环境, 允许在执行命令后马上得到结果.  它在调试项目时十分有效.

所有的 `tf.keras` 模块构建 API 都兼容 eager execution.
Eager execution 同样支持 *子类化模块* 和 *自定义 layer*, 这样更加有效的帮助了调试前传代码. 

查看 [eager execution guide](https://www.tensorflow.org/guide/eager#build_a_model) 文档. 里边有使用 Keras 模型的案例. 

## Distribution

### Estimators

[Estimators](https://www.tensorflow.org/guide/estimators) API 用于在分布式环境下训练模型. 目的是在行业内用例, 例如分布式训练一个大型数据集, 模型结果可以应用于生产环境. 

`tf.keras.Model` 模型可以通过使用`tf.keras.estimator.model_to_estimator`方法, 将其类型转为`tf.estimator.Estimator` 对象来使用 `tf.estimator` API 
. See
[Creating Estimators from Keras models](https://www.tensorflow.org/guide/estimators#creating_estimators_from_keras_models).

In [0]:
model = tf.keras.Sequential([layers.Dense(10,activation='softmax'),
                          layers.Dense(10,activation='softmax')])

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

estimator = tf.keras.estimator.model_to_estimator(model)

Note: 启用 [eager execution](https://www.tensorflow.org/guide/eager) 来 debugging
[Estimator input functions](https://www.tensorflow.org/guide/premade_estimators#create_input_functions)
和检查数据.

### 多 GPU

`tf.keras` 模型可以通过
`tf.contrib.distribute.DistributionStrategy` 在多 GPU 下执行. 此 API 可以在几乎不修改现有代码的情况下支持多 GPU 的分布式训练.

当前, `tf.contrib.distribute.MirroredStrategy` 仅支持分布式策略. `MirroredStrategy` 在一个机器上使用收敛算法同步训练, 再通过 in-graph 复制. 想在 Keras 上使用
`DistributionStrategy` , 通过`tf.keras.estimator.model_to_estimator`方法将 `tf.keras.Model` 转换为一个
`tf.estimator.Estimator` 对象, 再训练 estimator.

下面的例子是在一个机器上的多个 GPU 之间分发一个 `tf.keras.Model`.

首先, 定义一个简单的 model:

In [0]:
model = tf.keras.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10,)))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = tf.train.GradientDescentOptimizer(0.2)

model.compile(loss='binary_crossentropy', optimizer=optimizer)
model.summary()

定义一个 *input pipeline*.  `input_fn`方法返回一个 `tf.data.Dataset` 对象, 用于将数据分发到多个设备中 -- 每个设备将处理 input batch 的一部分.

In [0]:
def input_fn():
  x = np.random.random((1024, 10))
  y = np.random.randint(2, size=(1024, 1))
  x = tf.cast(x, tf.float32)
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  dataset = dataset.repeat(10)
  dataset = dataset.batch(32)
  return dataset

接下来, 创建一个 `tf.estimator.RunConfig`, 用`tf.contrib.distribute.MirroredStrategy` 实例来设置 `train_distribute` 参数. 在创建`MirroredStrategy`时, 可以设置设备列表或设置`num_gpus`. 默认使用所有 GPU 设备 :

In [0]:
strategy = tf.contrib.distribute.MirroredStrategy()
config = tf.estimator.RunConfig(train_distribute=strategy)

将 Keras model 转换为 `tf.estimator.Estimator` 实例:

In [0]:
keras_estimator = tf.keras.estimator.model_to_estimator(
  keras_model=model,
  config=config,
  model_dir='/tmp/model_dir')

最后, 设置`input_fn` 和 `steps`参数, 训练 `Estimator` 实例 :

In [0]:
keras_estimator.train(input_fn=input_fn, steps=10)